In [130]:
%autosave 45
#!/usr/bin/python
# -*- coding: utf-8 -*-

import tweepy
import spacy
import csv
import json
import pandas as pd
import os

Autosaving every 45 seconds


In [10]:
# Twitter API credentials
credentialsPath = r'..\0_data\credentials'
with open(os.path.join(credentialsPath, 'twitter_credentials.json')) as cred_data:
    info = json.load(cred_data)
    consumer_key = info['CONSUMER_KEY']
    consumer_secret = info['CONSUMER_SECRET']
    access_key = info['ACCESS_KEY']
    access_secret = info['ACCESS_SECRET']

# Create the api endpoint
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
api = tweepy.API(auth)

In [129]:
# Convert to data frame and capture each column feature in an array
def toDataFrame(tweets):
    DataSet = pd.DataFrame()
    
    tweetsText = []
    for tweet in tweets:
            if 'retweeted_status' in  dir(tweet):
                tweetsText.append(tweet.retweeted_status.full_text.encode('utf-8'))                
            else:
                tweetsText.append(tweet.full_text.encode('utf-8'))
            
    DataSet['Text'] = [text for text in tweetsText]
    DataSet['User'] = [tweet.user.name.encode('utf-8') for tweet in tweets]
    
    tweetsImages = []
    for tweet in tweets:
        if 'media' in tweet.entities:
            for image in tweet.entities['media']:
                tweetsImages.append(image['media_url'])
        else:
            tweetsImages.append('')
            
    
    DataSet['Image Urls'] = [image for image in tweetsImages]
            
    tweetsLongitudes = []
    for tweet in tweets:
        if tweet.coordinates is not None:
            tweetsLongitudes.append(tweet.coordinates["coordinates"][0])
        else:
            tweetsLongitudes.append('')
    DataSet['Longitude'] = [longitude for longitude in tweetsLongitudes]
    
    tweetsLatitudes = []
    for tweet in tweets:
        if tweet.coordinates is not None:
            tweetsLatitudes.append(tweet.coordinates["coordinates"][1])
        else:
            tweetsLatitudes.append('')
    DataSet['Latitude'] = [latitude for latitude in tweetsLatitudes]
    
    return DataSet


# Specify the maximum number of tweets that you want to be extracted.
maximum_number_of_tweets_to_be_extracted = \
    int(input('Enter the number of tweets that you want to extract- '))

# Specify if you are looking for hashtags or mentions
# Note: mentions will get you exactly a mention and are case insensitive
# Note 2: hashtags are case insensitive and not required
# Example: datascience, Datascience, #datascience, and #Datascience will return tweets about data science
# Hashtags of course are explicit mentions of a term, but are not case sensitive on Twitter
tag_choice = input('Enter which you are searching for, a hashtag (#), or a mention (@)- ')

# Specify the term you want to scrape
mention = input('Enter the term you want to scrape- ')

results = []

for tweet_info in tweepy.Cursor(api.search, q=str(tag_choice) + mention,
                           tweet_mode='extended').items(maximum_number_of_tweets_to_be_extracted):
    results.append(tweet_info)
    
for tweet_info in tweepy.Cursor(api.search, q=str(tag_choice) + mention.upper(),
                           tweet_mode='extended').items(maximum_number_of_tweets_to_be_extracted):
    results.append(tweet_info)

data = toDataFrame(results)
outputPath = r'..\0_data\manual'
filePath = os.path.join(outputPath,'tweets_with_mention_' + mention + '.csv')
if not os.path.isfile(filePath):
    data.to_csv(filePath, index=False)
else:
    with open(filePath, 'a') as file:
        data.to_csv(file, index = False)
print ('Extracted ' + str(maximum_number_of_tweets_to_be_extracted) \
    + ' tweets with ' + str(tag_choice) + mention)

Enter the number of tweets that you want to extract- 10000
Enter which you are searching for, a hashtag (#), or a mention (@)- @
Enter the term you want to scrape- twitter


TweepError: Twitter error response: status code = 429

In [127]:
# Instantiate the clean text and user arrays
clean_texts = []
clean_users = []

# Clean the text data
for row in data.itertuples():
    text = row[1].decode('utf-8').encode('ascii', 'ignore').decode('ascii').strip()
    user = row[2].decode('utf-8').encode('ascii', 'ignore').decode('ascii').strip()
    
#     text_doc = nlp.make_doc(text)
#     tokens = [token for token in text_doc if not token.is_stop]
#     tokens = [token for token in tokens if token.text != ' ']    
#     tokenz_final = [token.text for token in tokens]
    
#     text = " ".join(tokenz_final).strip()
    text = text.replace('\n', '')
    
    clean_texts.append(text)
    clean_users.append(user)

In [128]:
clean_df = pd.DataFrame({'Clean Tweet':clean_texts, 'Clean Username':clean_users})

In [116]:
# Remove the usernames (and thus tweets) that use unicode (were coerced to empty strings by the previous step)
# This means there are no non-English tweets/usernames
indexNames = clean_df[(clean_df['Clean Username'] == '') | (clean_df['Clean Username'] == '@')].index

clean_df.drop(indexNames, inplace=True)

,Clean Tweet,Clean Username
0,#scubadiving #scuba #diving #diver #scubadiver...,Dive Bot 2.0
1,There Are More Fish In The Sea https://t.co/nk...,Tami Quigley
2,My kind of #fish https://t.co/OV7kR2jj7H,Erick Scarecrow
3,Get some grilling on your plate! The inhouse r...,Chateau Garli
4,Lionfish ... perfect #art for the #fishing #en...,Lois Bryan
5,Give a man a #fish and he will eat for a day. ...,CharlestonBoozeChef
6,What #crudo dreams are made of... #OpenBlue #C...,GoodGREENSF
7,Give a man a #fish and he will eat for a day. ...,Fishing Skills
8,Wow great meal today salad and fish/shrimp and...,Adam [TeEm] [ICN] [FAMafia]
9,Sauteed Tuna by THE HUSBAND #WINEsdish #foode...,Desiree Charmaine


In [119]:
outputPath = r'..\0_data\manual'
filePath = os.path.join(outputPath,'tweets_with_mention_' + mention + '.csv')
if not os.path.isfile(filePath):
    clean_df.to_csv(filePath, index=False)
else:
    with open(filePath, 'a') as file:
        clean_df.to_csv(file, index = False)